# [Paper labyrinth](https://www.codingame.com/training/medium/paper-labyrinth)

You are Alice and you must find the rabbit then go out of the Queen’s labyrinth of death as quickly as you can.

The labyrinth is made of thin walls, each wall is binary-coded in each cell:
- 1 is the the down wall
- 2 the left wall
- 4 the top wall
- 8 the right wall

If the wall is present, add its number to the cell.

For example, 10=8+2 in a cell where you stand means that there are walls on your left and on your right and that you can walk downwards and upwards.

This also means that one-way doors are not forbidden.

Look for instance at 10 5, if you are on 5, you can go on 10 but you can’t go back.

In fact, the cells are coded in hexadecimal, 10 is a.

---

The first simple labyrinth is this one:

``` text
________
|S    R|
‾‾‾‾‾‾‾‾
```

* 7=4+2+1, it’s the start cell on the left
* 0xd=13=8+4+1, it’s the rabbit cell on the right
* The other cells are 5=4+1

### Импорт необходимых модулей

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from icecream import ic

### Отображение лабиринта

In [ ]:
# Returns image with borders depending on the specified cell code
def get_cell_image(cell_code: str, cell_size=100):
    right, top, left, bottom = map(int, bin(int(cell_code, 16))[2:].zfill(4))

    line_thickness = 5
    borders_color = (255, 0, 255)
    transparency_color = (10, 10, 10)

    # Create a cell image filled with zeros
    cell_image = np.zeros((cell_size, cell_size, 3))

    # Right border
    cv2.line(
        cell_image,
        (cell_size, 0),
        (cell_size, cell_size),
        color=borders_color if right else transparency_color,
        thickness=line_thickness,
        lineType=cv2.LINE_AA,
    )

    # Top border
    cv2.line(
        cell_image,
        (0, 0),
        (cell_size, 0),
        color=borders_color if top else transparency_color,
        thickness=line_thickness,
        lineType=cv2.LINE_AA,
    )

    # Left border
    cv2.line(
        cell_image,
        (0, 0),
        (0, cell_size),
        color=borders_color if left else transparency_color,
        thickness=line_thickness,
        lineType=cv2.LINE_AA,
    )

    # Bottom border
    cv2.line(
        cell_image,
        (0, cell_size),
        (cell_size, cell_size),
        color=borders_color if bottom else transparency_color,
        thickness=line_thickness,
        lineType=cv2.LINE_AA,
    )

    return cell_image

In [ ]:
# Renders an image of the labyrinth, Alice and rabbit positions
def render_labyrinth(labyrinth, alice_pos, rabbit_pos, cell_size=100):
    labyrinth_width = len(labyrinth[0])
    labyrinth_height = len(labyrinth)

    # Create a labyrinth image filled with zeros
    labyrinth_image = np.zeros(
        (labyrinth_height * cell_size, labyrinth_width * cell_size, 3),
        dtype=np.uint8,
    )

    # Iterate over every labyrinth cell
    for y in range(labyrinth_height):
        for x in range(labyrinth_width):
            y_start = y * cell_size
            y_end = y_start + cell_size

            x_start = x * cell_size
            x_end = x_start + cell_size

            # render cell on the labyrinth image
            labyrinth_image[y_start:y_end, x_start:x_end] = get_cell_image(
                labyrinth[y][x],
                cell_size,
            )

    # Draw a circle for Alice position
    alice_color = (255, 0, 0)
    cv2.circle(
        labyrinth_image,
        (
            int((alice_pos[0] + 0.5) * cell_size),
            int((alice_pos[1] + 0.5) * cell_size),
        ),
        radius=cell_size // 5,
        color=alice_color,
        thickness=-1,
    )

    # Draw a circle for rabbit position
    rabbit_color = (0, 0, 255)
    cv2.circle(
        labyrinth_image,
        (
            int((rabbit_pos[0] + 0.5) * cell_size),
            int((rabbit_pos[1] + 0.5) * cell_size),
        ),
        radius=cell_size // 5,
        color=rabbit_color,
        thickness=-1,
    )

    return labyrinth_image

In [ ]:
x_start, y_start = 0, 0
x_rabbit, y_rabbit = 1, 1

labyrinth = [
    "e65c",
    "abea",
    "2519",
    "355d",
]

labyrinth_image = render_labyrinth(
    labyrinth,
    alice_pos=(x_start, y_start),
    rabbit_pos=(x_rabbit, y_rabbit),
)

plt.title("Labyrinth image")
plt.imshow(labyrinth_image, vmin=0, vmax=255)

### Вспомогательные алгоритмы

In [ ]:
from typing import Tuple


# Get cell border flags
def get_labyrinth_walls(labyrinth_cell: str) -> Tuple[bool, bool, bool, bool]:
    right, top, left, bottom = map(int, bin(int(labyrinth_cell, 16))[2:].zfill(4))
    return right, top, left, bottom


# Remove excessive moves from the path
def shorten_path(labyrinth, path, x_start, y_start):
    x, y = x_start, y_start

    maze_width = len(labyrinth[0])
    maze_height = len(labyrinth)

    new_path = []

    for move in path:
        
        right, top, left, bottom = get_labyrinth_walls(labyrinth[y][x])

        if move == "R":  # moving right
            if not right:
                x = min(x + 1, maze_width - 1)
                new_path.append(move)

                if len(new_path) >= 2 and new_path[-2] == "L":
                    new_path = new_path[:-2]

        elif move == "L":  # moving left
            if not left:
                x = max(x - 1, 0)
                new_path.append(move)

                if len(new_path) >= 2 and new_path[-2] == "R":
                    new_path = new_path[:-2]

        elif move == "U":  # moving up
            if not top:
                y = max(y - 1, 0)
                new_path.append(move)

                if len(new_path) >= 2 and new_path[-2] == "D":
                    new_path = new_path[:-2]

        elif move == "D":  # moving down
            if not bottom:
                y = min(y + 1, maze_height - 1)
                new_path.append(move)

                if len(new_path) >= 2 and new_path[-2] == "U":
                    new_path = new_path[:-2]

    return new_path

### Определение генетического алгоритма

In [ ]:
import random
from deap import base, creator, tools, algorithms


# Calculate the number of cells needed to reach the rabbit and return back to the start
def solve_labyrinth(labyrinth, x_start, y_start, x_rabbit, y_rabbit) -> Tuple[int, int]:

    # Get labyrinth size
    maze_width = len(labyrinth[0])
    maze_height = len(labyrinth)

    # Evaluating fitness function for the given path
    def evaluate(path):
        x, y = x_start, y_start

        # Execute all moves
        for move in path:
            right, top, left, bottom = get_labyrinth_walls(labyrinth[y][x])

            if move == "R":  # move right
                if not right:
                    x = min(x + 1, maze_width - 1)
            
            elif move == "L":  # move left
                if not left:
                    x = max(x - 1, 0)
            
            elif move == "U":  # move up
                if not top:
                    y = max(y - 1, 0)
            
            elif move == "D":  # move down
                if not bottom:
                    y = min(y + 1, maze_height - 1)

        # Return the distance between the path end point and target point
        return abs(x - x_rabbit) + abs(y - y_rabbit),

    # Find optimal path by minimizing it's fitness function
    def geneticSolve(
        fitness_func,
        pop_size=500,
        num_generations=100,
        verbose: bool = True,
    ):
        # Create class for minimizing fitness function
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # Create individual class representing the path in the maze
        creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()

        # Create a random direction generator
        toolbox.register("attr", random.choice, ["U", "D", "L", "R"])

        # Create a generator of paths with specified length
        toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            toolbox.attr,
            n=maze_width * maze_height * 2,
        )

        # Create a function for generating population of individuals, i.e., a set of random paths
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Define crossing and mutation operators for two paths
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)

        # Set fitness function
        toolbox.register("evaluate", fitness_func)

        # Define operator for selecting the best path
        toolbox.register("select", tools.selTournament, tournsize=3)

        # Create an initial population
        population = toolbox.population(n=pop_size)

        # Launch the genetic algorithm
        population, logbook = algorithms.eaSimple(
            population,
            toolbox,
            cxpb=0.5,
            mutpb=0.2,
            ngen=num_generations,
            verbose=False,
        )

        # Select the best path in the last population
        best_individ = tools.selBest(population, k=1)[0]

        if verbose:
            print(f"Best path: {best_individ}")
            print(f"Distance to the target: {best_individ.fitness.values[0]}")

        # Return best path and it's distance to the target
        return best_individ, best_individ.fitness.values[0]

    return geneticSolve(fitness_func=evaluate, pop_size=1000, num_generations=100)

### Демонстрация работы алгоритма

In [ ]:
from enum import Enum

class PathMode(Enum):
    FROM_START_TO_THE_RABBIT = 1
    FROM_RABBIT_TO_THE_START = 2

In [ ]:
x_start, y_start = 0, 0
x_rabbit, y_rabbit = 1, 1

labyrinth = [
    "e65c",
    "abea",
    "2519",
    "355d",
]

In [ ]:
best_path_to_rabbit, best_path_to_rabbit_value = solve_labyrinth(
    labyrinth, x_start, y_start, x_rabbit, y_rabbit
)

best_path_to_start, best_path_to_start_value = solve_labyrinth(
    labyrinth, x_rabbit, y_rabbit, x_start, y_start
)

best_path_to_rabbit = shorten_path(labyrinth, best_path_to_rabbit, x_start, y_start)
best_path_to_start = shorten_path(labyrinth, best_path_to_start, x_rabbit, y_rabbit)

In [ ]:
ic(best_path_to_rabbit, best_path_to_start)

In [ ]:
%matplotlib inline
%matplotlib widget

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider

# Set path for displaying
path = best_path_to_rabbit
path_mode = PathMode.FROM_START_TO_THE_RABBIT

# Set Alice and rabbit positions
alice_pos = (x_start, y_start)
rabbit_pos = (x_rabbit, y_rabbit)

# Get labyrinth size
maze_width = len(labyrinth[0])
maze_height = len(labyrinth)

# Get Alice and rabbit positions
x_start, y_start = alice_pos
x_rabbit, y_rabbit = rabbit_pos

# Render an initial labyrinth image
labyrinth_images = [
    render_labyrinth(
        labyrinth,
        alice_pos=alice_pos,
        rabbit_pos=rabbit_pos,
    )
]

# Create a subplot for displaying current labyrinth image
mpl_fig, mpl_ax = plt.subplots(figsize=(12, 7))
mpl_fig.suptitle('Labyrinth path visualization', fontsize=16)

# Changing position of Alice with each move
if path_mode == PathMode.FROM_START_TO_THE_RABBIT:

    x, y = x_start, y_start

    # Execute each move in the path
    for move in path:

        right, top, left, bottom = get_labyrinth_walls(labyrinth[y][x])

        if move == "R":  # move right
            if not right:
                x = min(x + 1, maze_width - 1)

        elif move == "L":  # move left
            if not left:
                x = max(x - 1, 0)

        elif move == "U":  # move up
            if not top:
                y = max(y - 1, 0)

        elif move == "D":  # move down
            if not bottom:
                y = min(y + 1, maze_height - 1)

        # Render current labyrinth image
        labyrinth_image = render_labyrinth(
            labyrinth,
            alice_pos=(x, y),
            rabbit_pos=rabbit_pos,
        )

        labyrinth_images.append(labyrinth_image)

# Changing position of rabbit with each move
elif path_mode == PathMode.FROM_RABBIT_TO_THE_START:

    x, y = x_rabbit, y_rabbit

    # Execute each move in the path
    for move in path:

        right, top, left, bottom = get_labyrinth_walls(labyrinth[y][x])

        if move == "R":  # move right
            if not right:
                x = min(x + 1, maze_width - 1)

        elif move == "L":  # move left
            if not left:
                x = max(x - 1, 0)

        elif move == "U":  # move up
            if not top:
                y = max(y - 1, 0)

        elif move == "D":  # move down
            if not bottom:
                y = min(y + 1, maze_height - 1)

        # Render current labyrinth image
        labyrinth_image = render_labyrinth(
            labyrinth,
            alice_pos=alice_pos,
            rabbit_pos=(x, y),
        )

        labyrinth_images.append(labyrinth_image)

# Create a slider for selecting a step of the path
slider_ax = mpl_fig.add_axes([0.3, 0.0, 0.4, 0.05]) # [left, bottom, width, height]
slider = Slider(
    ax=slider_ax,
    label='Select step of the path',
    valmin=0,
    valmax=10,
    valinit=0,
    valstep=1,
)

# Define callback for the slider
def on_slider_changed(val):
    mpl_ax.clear()
    mpl_ax.set_title(f"Step #{val} of the path: {path}")

    mpl_ax.imshow(labyrinth_images[val])

# Render the initial image of the labyrinth
on_slider_changed(slider.val)

# Connect callback with the slider event
slider.on_changed(on_slider_changed)

plt.show()

### Тестирование алгоритма

In [ ]:
import unittest


class TestLabyrinth(unittest.TestCase):
    def setUp(self) -> None:
        super().setUp()

    def test_simple(self):
        self.x_start, self.y_start = 0, 0
        self.x_rabbit, self.y_rabbit = 5, 0

        self.labyrinth = ["75555d"]

        self.expected_turns_to_reach_rabbit = 5
        self.expected_turns_to_start = 5

    def test_no_one_way_door_no_loops(self):
        self.x_start, self.y_start = 0, 0
        self.x_rabbit, self.y_rabbit = 1, 1

        self.labyrinth = [
            "e65c",
            "abea",
            "2519",
            "355d",
        ]

        self.expected_turns_to_reach_rabbit = 10
        self.expected_turns_to_start = 10

    def test_one_way_door(self):
        self.x_start, self.y_start = 0, 0
        self.x_rabbit, self.y_rabbit = 1, 3

        self.labyrinth = [
            "75ce",
            "6598",
            "2758",
            "b759",
        ]

        self.expected_turns_to_reach_rabbit = 12
        self.expected_turns_to_start = 8
    
    def test_house_of_cards(self):
        self.x_start, self.y_start = 0, 0
        self.x_rabbit, self.y_rabbit = 3, 5

        self.labyrinth = [
            "7554e",
            "6459b",
            "a3c2c",
            "3caaa",
            "6119a",
            "35d39",
        ]
    
        self.expected_turns_to_reach_rabbit = 18
        self.expected_turns_to_start = 8

    def tearDown(self) -> None:
        best_path_to_rabbit, best_path_to_rabbit_value = solve_labyrinth(
            self.labyrinth, self.x_start, self.y_start, self.x_rabbit, self.y_rabbit
        )
        best_path_to_start, best_path_to_start_value = solve_labyrinth(
            self.labyrinth, self.x_rabbit, self.y_rabbit, self.x_start, self.y_start
        )

        best_path_to_rabbit = shorten_path(
            self.labyrinth, best_path_to_rabbit, self.x_start, self.y_start
        )
        best_path_to_start = shorten_path(
            self.labyrinth, best_path_to_start, self.x_rabbit, self.y_rabbit
        )

        turns_to_reach_rabbit, turns_to_start = len(best_path_to_rabbit), len(
            best_path_to_start
        )

        self.assertEqual(
            turns_to_reach_rabbit,
            self.expected_turns_to_reach_rabbit,
            f"Path to the rabbit: {best_path_to_rabbit}",
        )
        self.assertEqual(
            turns_to_start,
            self.expected_turns_to_start,
            f"Path to start: {best_path_to_start}",
        )


unittest.main(argv=[""], verbosity=2, exit=False)